In [1]:
import pandas as pd
import os

import sys
src_root = os.path.abspath(os.path.join(os.getcwd(), '../src'))
if src_root not in sys.path:
    sys.path.append(src_root)

In [2]:
# Load the external dataset
def load_data(filepath, name):
    df = pd.read_csv(filepath)
    print(f"{name} data loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")
    return df

# Model Data Analysis

## Filter Best Models

In [3]:
from itertools import product

data = {}
for (scale, preprocessing, data_type) in product(['standardized', 'normalized'], ['preprocessed', 'non_processed'], ['external', 'augmented']):
    df = load_data(f"../reports/results/training/{scale}/non_engineered/{preprocessing}/{data_type}/cv_history.csv", scale+' '+preprocessing+' '+data_type)

    data[(scale, preprocessing, data_type)] = df

standardized preprocessed external data loaded successfully with 6000 rows and 13 columns.
standardized preprocessed augmented data loaded successfully with 6000 rows and 13 columns.
standardized non_processed external data loaded successfully with 6000 rows and 13 columns.
standardized non_processed augmented data loaded successfully with 6000 rows and 13 columns.
normalized preprocessed external data loaded successfully with 6000 rows and 13 columns.
normalized preprocessed augmented data loaded successfully with 6000 rows and 13 columns.
normalized non_processed external data loaded successfully with 6000 rows and 13 columns.
normalized non_processed augmented data loaded successfully with 6000 rows and 13 columns.


In [4]:
from mda import filter_models

### External Data

In [5]:
for (scale, preprocessing, data_type) in product(['standardized', 'normalized'], ['preprocessed', 'non_processed'], ['external']):
    optimal_models = filter_models(data[(scale, preprocessing, data_type)])

    print("Best epochs per model:")
    print(optimal_models)

    optimal_models.to_csv(f"../reports/results/training/{scale}/non_engineered/{preprocessing}/{data_type}/optimal_models.csv", index=False)

Best epochs per model:
            layers activation  batch_size  epoch      loss       mae  \
223           [16]       relu          32    224  0.243459  0.255285   
501           [16]       relu          64    202  0.245398  0.256097   
814           [16]      prelu          32    215  0.242237  0.255664   
1147          [16]      prelu          64    248  0.241833  0.255468   
1497          [32]       relu          32    298  0.237544  0.253197   
1792          [32]       relu          64    293  0.236376  0.252511   
2070          [32]      prelu          32    271  0.237165  0.253005   
2390          [32]      prelu          64    291  0.235855  0.253284   
2672          [64]       relu          32    273  0.230161  0.249906   
2982          [64]       relu          64    283  0.231417  0.250355   
3297          [64]      prelu          32    298  0.230252  0.250754   
3585          [64]      prelu          64    286  0.231097  0.250290   
3876      [32, 16]       relu          32

### Augmented Data

In [6]:
for (scale, preprocessing, data_type) in product(['standardized', 'normalized'], ['preprocessed', 'non_processed'], ['augmented']):
    optimal_models = filter_models(data[(scale, preprocessing, data_type)])

    print("Best epochs per model:")
    print(optimal_models)

    optimal_models.to_csv(f"../reports/results/training/{scale}/non_engineered/{preprocessing}/{data_type}/optimal_models.csv", index=False)

Best epochs per model:
            layers activation  batch_size  epoch      loss       mae  \
242           [16]       relu          32    243  0.249227  0.258006   
583           [16]       relu          64    284  0.250013  0.258622   
822           [16]      prelu          32    223  0.250192  0.258921   
1192          [16]      prelu          64    293  0.248061  0.257720   
1412          [32]       relu          32    213  0.246026  0.256713   
1758          [32]       relu          64    259  0.246255  0.256716   
2085          [32]      prelu          32    286  0.245298  0.256868   
2360          [32]      prelu          64    261  0.244786  0.256448   
2645          [64]       relu          32    246  0.242958  0.255763   
2946          [64]       relu          64    247  0.243095  0.255664   
3248          [64]      prelu          32    249  0.242028  0.255278   
3585          [64]      prelu          64    286  0.241518  0.254942   
3884      [32, 16]       relu          32